# Imports

In [1]:
import gradio as gr

from datasets import load_dataset
from transformers import pipeline

# Data loading

In [2]:
dataset = load_dataset("allocine", split="test")["review"]

Found cached dataset allocine (/Users/sofiene.alouini/.cache/huggingface/datasets/allocine/allocine/1.0.0/ea86b1dc05eae3a45a07b6281f2d4033b5fe7927b1008d06aa457ca1eae660d0)


In [3]:
indices = [0, 16, 22, 23]

examples = [dataset[i] for i in indices]

for i in indices:
    print(f"=== Exemple {i} ===")
    print(dataset[i])
    print()

=== Exemple 0 ===
Magnifique épopée, une belle histoire, touchante avec des acteurs qui interprètent très bien leur rôles (Mel Gibson, Heath Ledger, Jason Isaacs...), le genre de film qui se savoure en famille! :)

=== Exemple 16 ===
Eli Roth est un réalisateur intéressant et redoutable d'efficacité. Bruce Willis fidèle à lui-même, le film ne révolutionne pas le genre mais il a son charme et pose de bonnes questions.

=== Exemple 22 ===
Alors oui on est loin du "Parrain" très loin de "Apocalypse Now" mais on le regarde en partie parce que c'est Coppola à la réalisation. Sans s'attendre à un chef d'oeuvre, on en reste pas moins conquit par ce film. L’esthétique de Poe (Notamment sa nouvelle "la Maison Usher") mêlé à la mélancolie Baudelerienne nous fait oublié le pas pataud et hésitant de Val Kilmer miné par quelques kilos en trop ( Bien que sont jeu d'acteur reste tout a fait correct). Coppola revient à une atmosphère plus gothique comme l'était sa vision très personnel du Dracula de B

# Pretrained model loading

In [4]:
pipe = pipeline("ner", "Jean-Baptiste/camembert-ner-with-dates")

In [5]:
pipe(examples[0])

[{'entity': 'I-PER',
  'score': 0.99518096,
  'index': 22,
  'word': 'M',
  'start': 105,
  'end': 106},
 {'entity': 'I-PER',
  'score': 0.99514526,
  'index': 23,
  'word': 'el',
  'start': 106,
  'end': 108},
 {'entity': 'I-PER',
  'score': 0.9953176,
  'index': 24,
  'word': '▁Gi',
  'start': 108,
  'end': 111},
 {'entity': 'I-PER',
  'score': 0.9953532,
  'index': 25,
  'word': 'b',
  'start': 111,
  'end': 112},
 {'entity': 'I-PER',
  'score': 0.9953011,
  'index': 26,
  'word': 'son',
  'start': 112,
  'end': 115},
 {'entity': 'I-PER',
  'score': 0.9950518,
  'index': 28,
  'word': '▁Heat',
  'start': 116,
  'end': 121},
 {'entity': 'I-PER',
  'score': 0.9951816,
  'index': 29,
  'word': 'h',
  'start': 121,
  'end': 122},
 {'entity': 'I-PER',
  'score': 0.9952442,
  'index': 30,
  'word': '▁Le',
  'start': 122,
  'end': 125},
 {'entity': 'I-PER',
  'score': 0.9952088,
  'index': 31,
  'word': 'd',
  'start': 125,
  'end': 126},
 {'entity': 'I-PER',
  'score': 0.9950905,
  'index

# Post-processing function

In [6]:
def ner(text):
    output = pipe(text)
    if len(output) == 0:
        pass
    else:
        res = []
        merged = output[0]
        for i in range(1, len(output)):
            _next = output[i]
            if (_next["start"] != merged["end"]) or (_next['entity'].startswith('B')):
                merged["entity"] = merged["entity"].split("-")[-1]
                res.append(merged)
                merged = _next
            else:
                merged = {
                    'entity': merged["entity"],
                    'score': merged["score"],
                    'index': merged["index"],
                    'word': merged["word"] + _next["word"],
                    'start': merged["start"],
                    'end': _next["end"]
                }
        merged["entity"] = merged["entity"].split("-")[-1]
        res.append(merged)
        output = res
    return {"text": text, "entities": output}    

In [7]:
ner(examples[0])["entities"]

[{'entity': 'PER',
  'score': 0.99518096,
  'index': 22,
  'word': 'Mel▁Gibson',
  'start': 105,
  'end': 115},
 {'entity': 'PER',
  'score': 0.9950518,
  'index': 28,
  'word': '▁Heath▁Ledger',
  'start': 116,
  'end': 129},
 {'entity': 'PER',
  'score': 0.9949374,
  'index': 34,
  'word': '▁Jason▁Isaacs',
  'start': 130,
  'end': 143}]

# Demo

In [8]:
demo = gr.Interface(ner,
                    gr.Textbox(placeholder="Écrire une phrase ici..."), 
                    gr.HighlightedText(),
                    examples=examples, 
                    allow_flagging="never")

demo.launch(height=950)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x1060fcd00>, 'http://127.0.0.1:7860/', None)